# Previsão de Alta Complexidade de casos SAR

SAR (Sindrome aguda respiratória) 

# Imports

In [1]:
import pandas as pd 

import pandas as pd
import math
import numpy as np
import inflection
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import xgboost as xgb
import datetime

## Funções Auxiliares

In [2]:
# mostrar todas as colunas
pd.set_option('display.max_columns', None)

# Descrição dos Dados

## Carga dos Dados

In [3]:
# Lendo CSV 
df = pd.read_csv('C:/Users/Dell/Documents/Modelos/sar/csv/alto_risco_sagr.csv', engine="python", sep=',', quotechar='"', error_bad_lines=False)

In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] 

In [5]:
df.shape 

(52653, 41)

## Renomear Colunas

In [6]:
cols_old = ['DT_NOTIFIC', 'SEM_NOT', 'DT_SIN_PRI', 'SEM_PRI', 'SG_UF_NOT',
       'ID_MUNICIP', 'CS_SEXO', 'NU_IDADE_N', 'CS_GESTANT', 'CS_RACA',
       'CS_ZONA', 'NOSOCOMIAL', 'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA',
       'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'CARDIOPATI',
       'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC',
       'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'VACINA', 'ANTIVIRAL',
       'RAIOX_RES', 'PCR_RESUL', 'EVOLUCAO', 'DOR_ABD', 'FADIGA', 'PERD_OLFT',
       'PERD_PALA', 'VACINA_COV']
snakecase = lambda x: inflection.underscore(x) 
cols_new = list(map(snakecase, cols_old)) 
#rename 
df.columns = cols_new 

## Dimensão Dataset

In [8]:
print('Numero de filas: {}'.format(df.shape[0])) 
print('Numero de colunas: {}'.format(df.shape[1])) 
print("\nOs Targets representam {:.2f}% do dataset.\n".format((df[df.evolucao == 1].shape[0] / df.shape[0]) * 100))

Numero de filas: 52653
Numero de colunas: 41

Os Targets representam 27.60% do dataset.



## Tipo Dados

In [12]:
df.dtypes 

dt_notific    datetime64[ns]
sem_not                int64
dt_sin_pri    datetime64[ns]
sem_pri                int64
sg_uf_not             object
id_municip            object
cs_sexo               object
nu_idade_n             int64
cs_gestant             int64
cs_raca                int64
cs_zona              float64
nosocomial           float64
febre                float64
tosse                float64
garganta             float64
dispneia             float64
desc_resp            float64
saturacao            float64
diarreia             float64
vomito               float64
cardiopati           float64
hematologi           float64
sind_down            float64
hepatica             float64
asma                 float64
diabetes             float64
neurologic           float64
pneumopati           float64
imunodepre           float64
renal                float64
obesidade            float64
vacina               float64
antiviral            float64
raiox_res            float64
pcr_resul     

## Modificando Tipo Data

In [11]:
df['dt_notific'] = pd.to_datetime(df['dt_notific'])
df['dt_sin_pri'] = pd.to_datetime(df['dt_sin_pri'])


## Identificando NA 

In [13]:
df.isnull().sum()

dt_notific        0
sem_not           0
dt_sin_pri        0
sem_pri           0
sg_uf_not         0
id_municip        0
cs_sexo           0
nu_idade_n        0
cs_gestant        0
cs_raca           0
cs_zona        3043
nosocomial     9787
febre          6075
tosse          3926
garganta      11483
dispneia       3544
desc_resp      6979
saturacao      4794
diarreia      11500
vomito        11810
cardiopati    24902
hematologi    29325
sind_down     29395
hepatica      29405
asma          28975
diabetes      26315
neurologic    29023
pneumopati    28939
imunodepre    29236
renal         28992
obesidade     28368
vacina        13842
antiviral      9349
raiox_res     16323
pcr_resul      4500
evolucao          0
dor_abd       12656
fadiga        11009
perd_olft     12302
perd_pala     12282
vacina_cov    12537
dtype: int64

In [ ]:
# Dimensão

df_total.shape

In [ ]:
df_total['EVOLUCAO'].unique()

In [ ]:
# Apagando filas

df_total.drop(df_total.loc[df_total['EVOLUCAO']=='13/01/2022'].index, inplace=True)
df_total.drop(df_total.loc[df_total['EVOLUCAO']=='25/11/2021'].index, inplace=True)
df_total.drop(df_total.loc[df_total['EVOLUCAO']=='NECA"'].index, inplace=True)

In [ ]:
# Olhando os dados da coluna EVOLUCAO

df_total['EVOLUCAO'].unique()

In [ ]:
# Convertemos pra númerico

df_total['EVOLUCAO'] = pd.to_numeric(df_total['EVOLUCAO']) 

In [ ]:
# Filtrando variável alvo

df = df_total.loc[(df_total.EVOLUCAO==1) | (df_total.EVOLUCAO==2)]

In [ ]:
df.shape

In [ ]:
df['EVOLUCAO'].unique()

In [ ]:
# reemplazando dados

df['EVOLUCAO'] = df['EVOLUCAO'].map({1.:0, 2.:1,},
                             na_action=None)

In [ ]:
# Proporção das clases

print(df.EVOLUCAO.value_counts())
print("\nOs Targets representam {:.2f}% do dataset.\n".format((df[df.EVOLUCAO == 1].shape[0] / df.shape[0]) * 100))

In [ ]:
df.info()

In [ ]:
# Seleção de Colunas

cols_drop = ['CO_REGIONA', 'CO_MUN_NOT','CO_UNI_NOT','TP_IDADE','COD_IDADE',
              'CO_PAIS','CO_RG_RESI','CO_MUN_RES','LOTE_REF','LAB_PR_COV',
              'LOTE_1_COV','LOTE_2_COV','FNT_IN_COV', 
              'ID_UNIDADE', 'CO_UNI_NOT', 'ID_REGIONA', 'CO_REGIONA', 'TP_IDADE',                 
              'ID_PAIS', 'CO_PAIS', 'OUT_ANIM', 'OUTRO_DES', 'ID_RG_INTE', 'CO_RG_INTE', 'ID_MN_INTE', 'CO_MU_INTE',
              'DT_SAIDUTI','TOMO_OUT', 'AMOSTRA', 'DT_COLETA', 'TP_AMOSTRA', 'OUT_AMOST',  'TP_TES_AN',
              'DT_RES_AN', 'RES_AN','POS_AN_OUT', 'AN_SARS2', 'AN_VSR', 'AN_PARA1','AN_PARA2',
              'AN_PARA3', 'AN_ADENO', 'AN_OUTRO', 'DS_AN_OUT', 'DT_PCR','TP_FLU_PCR']

In [ ]:
df1 = df.drop(cols_drop, axis=1) 

In [ ]:
df1.shape 

In [ ]:
# Limitar ao estado de PE e MG
df_states = df1.loc[(df1.SG_UF_NOT=='PE')|(df1.SG_UF_NOT=='MG')] 

In [ ]:
df_states.shape 

In [ ]:
# Proporção das clases

print(df_states.EVOLUCAO.value_counts())
print("\nOs Targets representam {:.2f}% do dataset.\n".format((df_states[df_states.EVOLUCAO == 1].shape[0] / df_states.shape[0]) * 100))

In [ ]:
# Guardar CSV filtrado

#df_states.to_csv('alta_complexidade_sagr.csv')


In [ ]:
# Carregando CSV

df_filtrado = pd.read_csv('alta_complexidade_sagr.csv')

In [ ]:
df_filtrado.shape

In [ ]:
df_filtrado.head()

In [ ]:
cols_drop = ['DT_NASC', 'SG_UF', 'ID_RG_RESI', 'ID_MN_RESI', 'DOSE_2_COV', 
             'DOSE_REF', 'FAB_COV_1', 'FAB_COV_2', 'FAB_COVREF','RES_IGM', 'AVE_SUINO', 
             'RES_IGA', 'DOSE_1_COV', 'OUT_SOR', 'DT_RES', 'RES_IGG', 'SOR_OUT', 'CS_ESCOL_N',
             'DT_CO_SOR', 'TP_SOR','TP_FLU_AN', 'POS_AN_FLU', 'TP_AM_SOR','TOMO_RES', 'DT_TOMO','ESTRANG', 'PCR_FLUBLI'
              ,'FLUBLI_OUT','PCR_PARA1','PCR_PARA2','PCR_PARA3','PCR_PARA4','PCR_ADENO','PCR_METAP','PCR_BOCA','PCR_RINO'
              ,'PCR_OUTRO','DS_PCR_OUT','CLASSI_OUT','PAIS_VGM','CO_PS_VGM','LO_PS_VGM','DT_VGM','DT_RT_VGM','PAC_COCBO','PAC_DSCBO',
             'SURTO_SG','OBES_IMC','MORB_DESC','DT_UT_DOSE','MAE_VAC','MAE_VAC','M_AMAMENTA','DT_DOSEUNI','DT_1_DOSE','DT_2_DOSE','TP_ANTIVIR','OUT_ANTIV',
              'DT_ANTIVIR','DT_ENTUTI','RAIOX_OUT','PCR_FLUASU','FLUASU_OUT','PCR_VSR','DT_VAC_MAE','PUERPERA']

In [ ]:
df_f1 = df_filtrado.drop(cols_drop, axis=1) 

In [ ]:
df_f1 = df_f1.loc[:, ~df_f1.columns.str.contains('^Unnamed')]

In [ ]:
df_f1.shape

In [ ]:
df_f1.head()

In [ ]:
# Proporção das clases

print(df_f1.EVOLUCAO.value_counts())
print("\nOs Targets representam {:.2f}% do dataset.\n".format((df_f1[df_f1.EVOLUCAO == 1].shape[0] / df_f1.shape[0]) * 100))

In [ ]:
df_f1.info()

In [ ]:
df_f1.isnull().sum()

In [ ]:
df_filtradox = pd.read_csv('alta_complexidade_sagr001.csv')

In [ ]:
df_f2 = df_filtradox.loc[:, ~df_filtradox.columns.str.contains('^Unnamed')]

In [ ]:
df_f2.shape

In [ ]:
df_filtradox.info()

In [ ]:
df_f2.to_csv('alta_complexidade_sagr001.csv')